In [3]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['대화 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess


#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            #### data preprocess
            line_data = json.loads(line)
            dialog_id = line_data['id']+'.1.'
            line = line.replace(dialog_id, '')
            line = json.loads(line)
            title = line['metadata']['title']
            date = line['metadata']['date']
            topic = line['metadata']['topic']
            speaker = line['metadata']['speaker']
            utterances = line['sentence']
            for utt in utterances:
                utt.pop('word')
                
            main_summary = line['SC']['main_summary']
            issue_summary = line['SC']['issue_summary']
            for issue in issue_summary:
                issue['issue'].pop('begin')
                issue['issue'].pop('end')
                
            #### data preprocess end 
            
            #### 대화 요약
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            data['input']['title'] = title
            data['input']['date'] = date
            data['input']['topic'] = topic
            data['input']['speaker'] = speaker
            data['input']['utterance'] = utterances
            data['output']['main_summary'] = main_summary
            data['output']['issue_summaries'] = issue_summary
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 대화 요약 end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train: 100%|██████████| 1/1 [00:05<00:00,  5.05s/it]
valid: 0it [00:00, ?it/s]
